<a href="https://colab.research.google.com/github/ibn-e-zulfiqar/British-airline-internship-/blob/main/airline_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Scrap and save in data.txt

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

urls=['https://www.airlinequality.com/airline-reviews/british-airways/?sortby=post_date%3ADesc&pagesize=100',
      'https://www.airlinequality.com/airline-reviews/british-airways/page/2/?sortby=post_date%3ADesc&pagesize=100',
      'https://www.airlinequality.com/airline-reviews/british-airways/page/3/?sortby=post_date%3ADesc&pagesize=100',
      'https://www.airlinequality.com/airline-reviews/british-airways/page/4/?sortby=post_date%3ADesc&pagesize=100',
      'https://www.airlinequality.com/airline-reviews/british-airways/page/5/?sortby=post_date%3ADesc&pagesize=100',
      'https://www.airlinequality.com/airline-reviews/british-airways/page/6/?sortby=post_date%3ADesc&pagesize=100',
      'https://www.airlinequality.com/airline-reviews/british-airways/page/7/?sortby=post_date%3ADesc&pagesize=100',
      'https://www.airlinequality.com/airline-reviews/british-airways/page/8/?sortby=post_date%3ADesc&pagesize=100',
      'https://www.airlinequality.com/airline-reviews/british-airways/page/9/?sortby=post_date%3ADesc&pagesize=100',
      'https://www.airlinequality.com/airline-reviews/british-airways/page/10/?sortby=post_date%3ADesc&pagesize=100',

]

with open("data.txt","w", encoding="utf-8") as f:
 for url in urls:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    reviews = soup.find_all("article", itemtype="http://schema.org/Review")
    f.write('\n'.join(str(r) for r in reviews) + '\n')




cleaning

In [ ]:
with open("data.txt", "r", encoding="utf-8") as file:
    html = file.read()

soup = BeautifulSoup(html, "html.parser")
reviews = soup.find_all("article")

def get_stat(review, label):
    tag = review.find("td", class_="review-rating-header " + label)
    if not tag:
        return "N/A"
    value_cell = tag.find_next_sibling("td")
    if value_cell is None:
        return "N/A"
    if "stars" in value_cell.get("class", []):
        return str(len(value_cell.find_all("span", class_="star fill")))
    return value_cell.text.strip()

records = []

for review in reviews:
    try:
        record = {
            "date_published": review.find("meta", itemprop="datePublished")["content"],
            "rating": review.find("span", itemprop="ratingValue").text.strip(),
            "title": review.find("h2", class_="text_header").text.strip(),
            "author": review.find("span", itemprop="name").text.strip(),
            "review_text": review.find("div", itemprop="reviewBody").text.strip(),
            "aircraft": get_stat(review, "aircraft"),
            "type_of_traveller": get_stat(review, "type_of_traveller"),
            "cabin_flown": get_stat(review, "cabin_flown"),
            "route": get_stat(review, "route"),
            "date_flown": get_stat(review, "date_flown"),
            "seat_comfort": get_stat(review, "seat_comfort"),
            "cabin_staff_service": get_stat(review, "cabin_staff_service"),
            "food_and_beverages": get_stat(review, "food_and_beverages"),
            "inflight_entertainment": get_stat(review, "inflight_entertainment"),
            "ground_service": get_stat(review, "ground_service"),
            "wifi_and_connectivity": get_stat(review, "wifi_and_connectivity"),
            "value_for_money": get_stat(review, "value_for_money"),
            "recommended": get_stat(review, "recommended"),
        }
        records.append(record)
    except Exception as e:
        print("Skipped a review due to error:", e)

# Save to Excel using pandas
df = pd.DataFrame(records)
df.to_excel("reviews.xlsx", index=False)

